# gif_grinder
A python script to serially create gifs from a movie file using a reference list.

gif_grinder is an over-script which parses a single flat text file that resides in the same directory as a movie file (avi, mkv, mpeg, etc.), to create a series of animated gifs based on the beginning and ending time values in said file.

The script utilizes https://github.com/Zulko/moviepy and https://github.com/FFmpeg/FFmpeg to carve out animated gifs from any movie file that FFmpeg can parse.

I typically grab the timestamps from vlc (http://www.videolan.org/vlc/index.html) using the Jump to time plugin "https://addons.videolan.org/content/show.php/Jump+to+time+(Previous+frame)+v2.1?content=156396".

The time length of the animated gifs is set by altering the "total_t" value in the script to you desired maximum length of time.  Times under 20 seconds are recommended as longer length gifs tend to create 100Mb+ files.

"gifspeed" can be changed to a float value ranging from 0.01 to 10+.  The smaller the number, the slower the gif.  Every above 1, multiplies the speed by that number.  2 is twice as fast. 5 is five times faster.

The text file format is as follows:

start - finish # cropstart_x, cropstart_y, crop_width, crop_height, speed
> 00:00:00:00.00 - 00:00:00:00.00<br>
> 00:00:00.00 - 00:00:00.00<br>
> 00:00.00 - 00:00.00<br>
> 00.00 - 00.00<br>
>
> 00:00.00 - 00:00.00 # 0000,0000,0000,0000,0.00<br>
> 00,00.00 - 00,00.00 # 0000,0000,0000,0000,0.00<br>

Leaving out the crop values is completely valid.

Expected time values are as follows:

> minutes:seconds.microseconds<br>
> minutes,seconds.microseconds

The gifs will be created using the name of the movie file with an incremental value appended.

> mymovie.mpg

creates:

> mymovie1.gif<br>
> mymovie2.gif

from a text file containing:

> 00:01.00 - 00:04.00<br>
> 00:06.01 - 00:10.15

If the calculated time value in your text file exceeds the maximum time (total_t), a warning message will be printed and creation of the gif will be skipped.

Currently the script looks for a file named "Untitled" as for the list of times and cropping.  It can be "Untitled Document", "Untitled file", or any other file where the name contains the exact word "Untitled".


*files with "gif" in the name (not just the suffix) will be skipped.
*files containing "ficache" or "fispool" will be deleted as the script assumes they were leftover from a previous failed run of the gif_grinder script.

For text overly to work properly you may need to edit your /etc/ImageMagic-6/policy.xml file.  Comment out the last policy domain line as below:

```
<!-- <policy domain="path" rights="none" pattern="@*" /> -->
```


In [14]:
%%python
#!/usr/bin/python

import os
from os import listdir
from os.path import isfile, join
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import *


mypath = os.getcwd()
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# assumes target filename is "Untitled"
# valid file contents
# start - finish # cropstart_x, cropstart_y, crop_width, crop_height, speed
#
# 00:00:00:00.00 - 00:00:00:00.00
# 00:00:00.00 - 00:00:00.00
# 00:00.00 - 00:00.00
# 00.00 - 00.00
#
# 00:00.00 - 00:00.00 # 0000,0000,0000,0000,0.00
# 00,00.00 - 00,00.00 # 0000,0000,0000,0000,0.00

file1 = onlyfiles[0]
file2 = onlyfiles[1]
# print '',file1, file2

# max video length in seconds
total_t = 80

contents = ''
cropped = 0
currentgifnumber = 0
scale = ''
videoname = ''

# speed up gif >1, slow down gif <1
# gifspeed = '0.5'
gifspeed = '1'


# convert time to seconds
def get_sec(s):
    remainder = s.split('.')
    # take the number of fields (i, x) and use that as the exponent. multiply 60 seconds by the exponent.  add remaining microseconds.
    return(sum(float(x) * 60 ** i for i, x in enumerate(reversed(remainder[0].split(":"))))) + (float(remainder[1]) * 0.001)


# convert to gif
def processGif():
    # print "st:", starttime
    # print "et:", endtime
    # print "video:", videoname
    # procvideoname = '"' + videoname + '"'
    # procvideoname = mypath + "/" + videoname

    # clips should be under total_t seconds
    totaltime = endtime - starttime
    if totaltime <= (total_t):

        # print "clip = VideoFileClip(%s).subclip((%s),(%s))" % (videoname, starttime, endtime)
        # print "crop = x_1(%s),y_1(%s),wdt(%s),hgt(%s)" % (x_1, y_1, wdt, hgt)

        # look for cropping information
        if cropped == 1:
            clip = crop(VideoFileClip(videoname).subclip((starttime), (endtime)), x1=(x_1), y1=(y_1), width=(wdt), height=(hgt))
        else:
            clip = VideoFileClip(videoname).subclip((starttime), (endtime))
        # if scale:
            # disabled, too slow
            # clip = resize(VideoFileClip(videoname), newsize=scale)
        # clip = VideoFileClip(procvideoname).subclip((01,59.60),(02,04.24))
        endfile = shortvideoname + "_" + str(currentgifnumber) + ".gif"
        # optendfile = shortvideoname + "_opt_" + str(currentgifnumber) + ".gif"
        # print "endfile:", endfile
        clip.subclip().speedx(float(gifspeed)).write_gif(endfile, loop=1)

        # post process compression
        # gifsicle = "gifsicle --colors 256 -O3 " + endfile + " -o " + optendfile
        # print "gifsicle:", gifsicle
        # gifsicle = ["/usr/bin/gifsicle /-/-colors 256 /-O3", endfile, "/-o", optendfile]
        # call(gifsicle)

    else:
        print("clip too long.")

# process directory listing
# blank lines are purged
# gif files are skipped
# temp files for moviepy are deleted


for i in onlyfiles:
#     print(("file:", i))
    processfile = i
    if "Untitled" in processfile:
        # print processfile
        contents = [contents.rstrip('\n') for contents in open(processfile)]
        # print("Contents1:", contents)
        # replace with generator?
        contents = [name for name in contents if name.strip()]
        # print("Contents2:", contents)
        # contents = filter(None, contents)
        # print("Contents3:", contents)
        # print contents
        # with open(processfile) as f:
        #     contents = f.readlines()
        totalgifs = len(contents)
        # print("total gifs:", totalgifs)
    elif ".gif" in processfile:
        print("")
    elif ".py" in processfile:
        print("")
    elif ".ipynb" in processfile:
        print("")
    elif ".md" in processfile:
        print("")
    elif "ficache" in processfile:
        os.unlink(processfile)
    elif "fispool" in processfile:
        os.unlink(processfile)
    else:
        videoname = i
        shortvideoname = i.split('.')[0]


# parse start end timestamps
for element in contents:
    currentgifnumber += 1
    if currentgifnumber <= totalgifs:
        if element.startswith("#"):
            print(("skipping line: ", currentgifnumber))
            print('')

        else:
            # print "element:", element

            if "#" in element:
                cropdata = element.split('#')
                crops = cropdata[-1].split(',')
                x_1 = int(crops[0])
                y_1 = int(crops[1])
                wdt = int(crops[2])
                hgt = int(crops[3])
                if len(crops) >= 5:
                    gifspeed = float(crops[4])
                else:
                    gifspeed = 1
                if len(crops) >= 6:
                    scale = float(crops[5])
                print(("x_1:", x_1, "y_1:", y_1, "wdt:", wdt, "hgt:", hgt, "speed:", gifspeed, scale))

                partsdata = cropdata[0].split('-')
                parts = partsdata
                cropped = 1

            else:
                parts = element.split("-")
                cropped = 0

            # print("element:", element)
            # print("parts:", parts)
            starttime0 = str(parts[0]).replace(",", ".").strip()
            endtime0 = str(parts[-1]).replace(",", ".").strip()
            print(("0starttime:", starttime0, "endtime:", endtime0))

            starttime = get_sec(starttime0)
            endtime = get_sec(endtime0)
            print(("1starttime:", starttime, "endtime:", endtime))

            print(("currentgifnumber:", currentgifnumber))
            # print "st:", starttime
            # print "et:", endtime
            # print "video:", videoname
            # print "shortvideoname:", shortvideoname
            # print "parts:", parts
            processGif()

# post process colorize
# convert 1.gif -fill green -tint 100 1b.gif

# use exifdata to get size of image
# do math to sort out image combination spacing and sizing
# convert green.gif -gamma 0.65 -black-threshold 10% green1.gif

# moviepy.video.fx.all.headblur(clip, fx, fy, r_zone, r_blur=None)
# Returns a filter that will blurr a moving part (a head ?) of the frames. The
# position of the blur at time t is defined by (fx(t), fy(t)), the radius of
# the blurring by r_zone and the intensity of the blurring by r_blur. Requires
# OpenCV for the circling and the blurring. Automatically deals with the case
# where part of the image goes offscreen.

# exiftool -ImageSize video.avi





('0starttime:', '00:05.000', 'endtime:', '00:08.000')
('1starttime:', 5.0, 'endtime:', 8.0)
('currentgifnumber:', 1)
MoviePy - Building file quatermass_and_the_pit_1.gif with imageio.


![SegmentLocal](quatermass_and_the_pit_1.gif "segment")
